In [ ]:
import scoring_functions as fn

#### Operation description @ SLM prompt with extra examples

How well different models generate an operation description for use in an OpenAPI spec, given an API request

| **Final Score** | **Interpretation** |
|--------------|------------------|
| **0.90 – 1.00** | 🔥 **Excellent match** (high accuracy, correct structure, no missing info) |
| **0.80 – 0.89** | ✅ **Good match** (minor variations, but correct meaning) |
| **0.70 – 0.79** | ⚠️ **Moderate match** (some incorrect phrasing or extra info) |
| **Below 0.70** | ❌ **Weak match** (significant deviation or missing context) |

In [ ]:
scenario = "Generate OpenAPI operation description using SLM prompt w/extra examples"
split_response = False
prompt_template = """
    **Prompt:**  
    Generate a concise operation summary for an OpenAPI specification using the
    following format:  

    1. Start with an imperative verb (e.g., "Retrieve", "Update", "Delete").  
    2. Clearly describe the purpose of the operation in 5-10 words.  
    3. Mention key elements from the URL and method, if relevant.

    Use the given request method and URL. Reply with only the summary.  

    Example:
    **Request:** `GET https://api.contoso.com/books/{{books-id}}`  
    **Summary:** Retrieve information about a specific book

    Example:
    **Request:** `GET https://api.contoso.com/books/{{books-id}}/authors`  
    **Summary:** Retrieve information about authors for a specific book

    Example:
    **Request:** `GET https://api.contoso.com/books/{{books-id}}/authors/{{authors-id}}`  
    **Summary:** Retrieve information about a specific author for a specific book

    Example:
    **Request:** `POST https://api.contoso.com/books/{{books-id}}/authors`  
    **Summary:** Add information about an author for a specific book

    Now, generate the operation summary for the following:
    **Request:** `{request}`
    **Summary:** 
    """.strip()
tasks = [
    {
        "name": "Retrieve a list of calendars for a specific user",
        "reference_texts": [
            "Retrieve a user's calendars",
            "Retrieve calendars for a specific user",
            "Retrieve a list of calendars for a specific user",
            "Fetch the list of calendars associated with a user"
        ],   
        "prompt": prompt_template.format(request="GET https://api.contoso.com/users/{{users-id}}/calendars")
    },
    {
        "name": "Retrieve information about a specific user",
        "reference_texts": [
            "Fetch details for the specified user",
            "Get user details",
            "Retrieve information about a specific user",
            "Retrieve details of a specific user",
            "Get information about a given user"
        ],
        "prompt": prompt_template.format(request="GET https://api.contoso.com/users/{{users-id}}")
    },
    {
        "name": "Retrieve information about a specific calendar for a given user",
        "reference_texts": [
            "Retrieve information about a specific calendar for a given user.",
            "Retrieve details of a specific calendar for a user",
            "Fetch information about a particular calendar owned by a user",
            "Get metadata for a specific calendar linked to a user",
            "Retrieve a user's calendar details",
            "Retrieve calendar details for a specific user",
            "Retrieve information about a specific user's calendar"
        ],
        "prompt": prompt_template.format(request="GET https://api.contoso.com/users/{{users-id}}/calendars/{{calendars-id}}")
    },
    {
        "name": "Create a new calendar for the specified user",
        "reference_texts": [
            "Create a new calendar for the specified user",
            "Add a calendar to the user's account",
            "Create a new calendar for a given user"
        ],
        "prompt": prompt_template.format(request="POST https://api.contoso.com/users/{{users-id}}/calendars")
    }
]
score_threshold = 0.8
metric_scorers = [
    {
        'name': 'BERT-F',
        'fn': fn.calculate_bert_f,
        'weight': 0.45
    },
    {
        'name': 'Edit distance',
        'fn': fn.calculate_edit_distance,
        'weight': 0.1
    },
    {
        'name': 'ROUGE-2',
        'fn': fn.calculate_rouge2,
        'weight': 0.25
    },
    {
        'name': 'ROUGE-L',
        'fn': fn.calculate_rougeL,
        'weight': 0.2
    }
]